In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install simplematrixbotlib
!{sys.executable} -m pip install "matrix-nio[e2e]"
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import simplematrixbotlib as botlib
import os
import requests
import json

In [ ]:
config = botlib.Config()
config.encryption_enabled = True
config.emoji_verify = True
config.ignore_unverified_devices = True
config.store_path = './crypto_store/'

In [ ]:
MATRIX_HOMESERVER = "https://matrix-client.matrix.org"   # change this if needed 
BOT_ACCOUNT_LOGIN = "BOT_ACCOUNT_LOGIN"                  # change this to bot account login 
BOT_ACCOUNT_PASSWORD = "BOT_ACCOUNT_PASSWORD"            # change this to bot account password 

In [ ]:
creds = botlib.Creds(MATRIX_HOMESERVER, BOT_ACCOUNT_LOGIN, BOT_ACCOUNT_PASSWORD)
bot = botlib.Bot(creds, config)

In [ ]:
if os.environ.get('DOCKER_ENV') == 'true':
    companion_host = 'host.docker.internal'
else:
    companion_host = 'localhost'


ai_companion_server = f"http://{companion_host}:3000"

BOT_NAME = "ai"
BOT_PERSONA = "ai is a artificial intelligence chatbot chatting with user"
FIRST_MESSAGE = "Hello user!"

In [ ]:
def ai_config():
    companionData = {
        "id": 0,
        "name": BOT_NAME,
        "persona": BOT_PERSONA,
        "first_message": FIRST_MESSAGE
    }
    response = requests.post(ai_companion_server+"/api/change/companionData",
                            json=companionData)
def ai_prompt(text):
    response = requests.post(ai_companion_server+"/api/prompt",
                            json={ "prompt": text })
    if response.status_code == 200:
        response_json = response.json()
        return response_json["text"]

In [ ]:
@bot.listener.on_message_event
async def echo(room, message):
    match = botlib.MessageMatch(room, message, bot)
    if match.is_not_from_this_bot():
        await bot.api.send_text_message(room.room_id, ai_prompt(message.body))

In [ ]:
if __name__ == "__main__":
    ai_config()
    bot.run()